# Alpha-AGI Business v1 👀✨

Run the introductory **Alpha-Factory** business demo. Works offline\nor upgrades automatically when `OPENAI_API_KEY` is set.\n\nThis notebook verifies requirements using `check_env.py --auto-install` and reads configuration from `config.env` (auto-created from the sample).

## 0 · Runtime check

In [ ]:
!nvidia-smi -L || echo 'GPU not detected - running on CPU'

## 1 · Clone repo & install dependencies

In [ ]:
%%bashset -eif [ ! -d AGI-Alpha-Agent-v0 ]; then  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.gitficp alpha_factory_v1/demos/alpha_agi_business_v1/config.env.sample alpha_factory_v1/demos/alpha_agi_business_v1/config.env 2>/dev/null || truecd AGI-Alpha-Agent-v0pip -q install openai_agents fastapi uvicorn gradio

In [ ]:
import sys
sys.path.append('AGI-Alpha-Agent-v0')
import check_env
try:
    check_env.main(['--auto-install'])
except Exception as e:
    print('Error: Environment check or dependency installation failed.')
    print(f'Details: {e}')
    raise


## 2 · (Optional) Configure your OpenAI API key

In [ ]:
import os, getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key (leave blank for offline): ')

### (Optional) Live market price feedSet `YFINANCE_SYMBOL` to fetch a current closing price using `yfinance`.

In [ ]:
import os
os.environ['YFINANCE_SYMBOL'] = os.getenv('YFINANCE_SYMBOL', 'SPY')

## 3 · Launch orchestrator

In [ ]:
import subprocess, threading, sys, time, requests
from IPython.display import display, IFrame

root = 'AGI-Alpha-Agent-v0/alpha_factory_v1/demos/alpha_agi_business_v1'
proc = subprocess.Popen([sys.executable, 'start_alpha_business.py', '--no-browser'],
                        cwd=root, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

def _tail():
    for line in proc.stdout:
        print(line, end='')

threading.Thread(target=_tail, daemon=True).start()

for _ in range(40):
    try:
        if requests.get('http://localhost:8000/healthz').status_code == 200:
            break
    except Exception:
        time.sleep(0.2)

display(IFrame(src='http://localhost:8000/docs', width='100%', height=600))


## 4 · Programmatic API call

In [ ]:
import requests
print('Available agents:', requests.get('http://localhost:8000/agents').json())

## 4b · Trigger Execution Agent

In [ ]:
import requests
requests.post('http://localhost:8000/agent/alpha_execution/trigger')
print('execution triggered')

## 5 · OpenAI Agents bridge

In [ ]:
%%bashALPHA_FACTORY_ENABLE_ADK=true python openai_agents_bridge.py --wait-secs 15 >/tmp/bridge.log 2>&1 &sleep 2tail -n 5 /tmp/bridge.log

## 5b · Interactive Gradio dashboard

In [ ]:
%%bash
python gradio_dashboard.py >/tmp/gradio.log 2>&1 &
sleep 2
tail -n 5 /tmp/gradio.log

In [ ]:
from IPython.display import IFrame
IFrame(src='http://localhost:7860', width='100%', height=480)

## 5bb · ADK gateway (optional)

In [ ]:
from IPython.display import IFrameIFrame(src="http://localhost:9000/docs", width="100%", height=320)

In [ ]:
# 5c · Query recent alpha via OpenAI Agents bridge
import requests
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'recent_alpha'})
if resp.status_code == 200:
    try:
        print('recent alpha:', resp.json())
    except ValueError as e:
        print('Error decoding JSON response:', e)
else:
    print(f'Error: Received status code {resp.status_code} with response: {resp.text}')

In [ ]:
# 5cc · Trigger risk assessment via OpenAI Agents bridge
import requests
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'risk'})
print('risk response:', resp.text)


In [ ]:
# 5d · Submit a custom job via the bridge
job = {'agent': 'alpha_execution', 'symbol': 'AAPL', 'qty': 1}
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'submit_job', 'job': job})
print('job submission:', resp.text)

In [ ]:
# 5e · Search memory via the bridge
import requests
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'search_memory', 'query':'supply chain', 'limit': 3})
print('memory search:', resp.text)

## 6 · Graceful shutdown

In [ ]:
proc.terminate(); print('✅ Orchestrator stopped')